<table width=100%>
<tr>
<td width=100%>
<h1><b>Master in Computer Vision - M6</b></h1>
<h2><b>Dense optical flow comparison</b></h2>
<h4>Ayan Banerjee under supervision of Josep Ramon Morros
<br>
<a href="https://imatge.upc.edu/web/"> GPI @ IDEAI</a> Research group
</h4>
</td>
</tr>
</table>

# Dense optical flow computation

Compute optical flow with several methods to compare the results. To run this demo:

 - First, change runtime type (Runtime --> Change runtime type) to use a GPU.

 - Then download the file of_comp.zip that you will find at the virtual campus and upload it here using the menu on the left (click the folder icon, upload the file). **Wait until the upload is complete, it may take a long time**

Install necessary packages



In [ ]:
!pip install recordclass
!pip install pyoptflow # Horn-Schunck implementation
!pip install flow_vis  # Install package for optical flow visualization using color coding
!pip install apng      # To create animated png's

In [ ]:
!git clone https://github.com/MaximKuklin/RAFT

In [ ]:
# Clone pyflow repository. For Brox method
!git clone https://github.com/pathak22/pyflow.git
# Build the package
%cd /content/pyflow/
!python setup.py build_ext -i
%cd ..

In [ ]:
!unzip of_comp.zip

In [ ]:
import os
import sys
import time

from argparse import ArgumentParser
from collections import OrderedDict

import cv2
import numpy as np
import torch
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

from flow_vis import flow_to_color

from pyoptflow import HornSchunck

In [ ]:
from of_comp.code.display_images import display_image, display_images
from of_comp.code.display_of import flow_with_legend

Download models for RAFT:

In [ ]:
%cd RAFT
!./download_models.sh

Import the RAFT packages:

In [ ]:
%cd /content/RAFT
sys.path.append('core')
from raft import RAFT
from utils.utils import InputPadder
from utils import flow_viz

Several helper functions for RAFT:

In [ ]:
def frame_preprocess(frame, device):
    frame = torch.from_numpy(frame).permute(2, 0, 1).float()
    frame = frame.unsqueeze(0)
    frame = frame.to(device)
    return frame

def load_image(imfile):
    img = np.array(Image.open(imfile)).astype(np.uint8)
    img = torch.from_numpy(img).permute(2, 0, 1).float()
    return img[None].to(DEVICE)

def get_cpu_model(model):
    new_model = OrderedDict()
    # get all layer's names from model
    for name in model:
        # create new name and update new model
        new_name = name[7:]
        new_model[new_name] = model[name]
    return new_model

Parse some options for RAFT:

In [ ]:
from argparse import ArgumentParser

DEVICE = "cuda"

parser = ArgumentParser()
parser.add_argument("--model", help="restore checkpoint")
parser.add_argument("--iters", type=int, default=12)
#parser.add_argument("--video", type=str, default="./videos/car.mp4")
parser.add_argument("--save", action="store_true", help="save demo frames")
parser.add_argument("--small", action="store_true", help="use small model")
parser.add_argument("--mixed_precision", action="store_true", help="use mixed precision")

#args = parser.parse_args(['--model', './models/raft-things.pth', '--video', '../crowd.mp4', '--iters', '12',  '--save'])
args = parser.parse_args(['--model', './models/raft-kitti.pth', '--iters', '12',  '--save'])

Define a function for computing the optical flow on two images

In [ ]:
def inference(ima1_name, ima2_name, args):
    # get the RAFT model
    model = RAFT(args)
    # load pretrained weights
    pretrained_weights = torch.load(args.model)

    save = args.save
    if save:
        if not os.path.exists("demo_frames"):
            os.mkdir("demo_frames")

    if torch.cuda.is_available():
        device = "cuda"
        # parallel between available GPUs
        model = torch.nn.DataParallel(model)
        # load the pretrained weights into model
        model.load_state_dict(pretrained_weights)
        model.to(device)
    else:
        device = "cpu"
        # change key names for CPU runtime
        pretrained_weights = get_cpu_model(pretrained_weights)
        # load the pretrained weights into model
        model.load_state_dict(pretrained_weights)

    # change model's mode to evaluation
    model.eval()
    with torch.no_grad():

        image1 = load_image(ima1_name)
        image2 = load_image(ima2_name)

        s = time.time()
        padder = InputPadder(image1.shape)
        image1, image2 = padder.pad(image1, image2)

        flow_low, flow_up = model(image1, image2, iters=20, test_mode=True)
        e = time.time()

        of = flow_up.detach().cpu().numpy()
        of = np.transpose(of,axes=(2,3,1,0))[:,:,:,0]

    return (of, e-s, flow_up)


Compute the optical flow on a pair of images:

In [ ]:
from apng import APNG                        # To create animated png's
from IPython.display import Image as IImage  # To display images in Colab

#image1_name = '/content/of_comp/images/vid_000340.png'
#image2_name = '/content/of_comp/images/vid_000341.png'
image1_name = '/content/of_comp/images/cd_000042.png'
image2_name = '/content/of_comp/images/cd_000046.png'

import cv2
ima1  = cv2.imread(image1_name)
ima2  = cv2.imread(image2_name)

ima1g = cv2.cvtColor(ima1, cv2.COLOR_BGR2GRAY)
ima2g = cv2.cvtColor(ima2, cv2.COLOR_BGR2GRAY)


APNG.from_files([image1_name, image2_name], delay=800).save("/content/result.png")
IImage('/content/result.png')

In [ ]:
%cd /content/RAFT
of_raft, tt, flow_up = inference(image1_name, image2_name, args)
print('Time Taken: {:.2f} seconds for image of size ({},{},{})'.format(tt, ima1.shape[0], ima1.shape[1], ima1.shape[2]))

flow_with_legend(of_raft)

Compute the optical flow witht he Horn-Schunck method:

In [ ]:
s = time.time()
U, V = HornSchunck(ima1g, ima2g, alpha=1.0, Niter=100)
e = time.time()
print('Time Taken: {:.2f} seconds for image of size ({},{},{})'.format(e - s, ima1.shape[0], ima1.shape[1], ima1.shape[2]))

of_hs = np.stack([U,V], axis=2)
flow_with_legend (of_hs)

Compute optical flow with the [Farneback](http://www.diva-portal.org/smash/get/diva2:273847/FULLTEXT01.pdf) method:

In [ ]:
s = time.time()
of_fb = cv2.calcOpticalFlowFarneback(ima1g,ima2g, None, 0.5, 3, 15, 3, 5, 1.2, 0)
e = time.time()
print('Time Taken: {:.2f} seconds for image of size ({},{},{})'.format(e - s, ima1.shape[0], ima1.shape[1], ima1.shape[2]))
flow_with_legend (of_fb)

Compute optical flow with the Brox method:

In [ ]:
# Flow Options:
alpha = 0.012
ratio = 0.75
minWidth = 20
nOuterFPIterations = 7
nInnerFPIterations = 1
nSORIterations = 30
colType = 0  # 0 or default:RGB, 1:GRAY (but pass gray image with shape (h,w,1))

In [ ]:
%cd /content/pyflow
import pyflow
import time
import numpy as np

s = time.time()
u, v, im2W = pyflow.coarse2fine_flow(
    ima1.astype(float)/255, ima2.astype(float)/255, alpha, ratio, minWidth, nOuterFPIterations, nInnerFPIterations,
    nSORIterations, colType)
of_brox = np.stack([u,v], axis=2)
e = time.time()
print('Time Taken: {:.2f} seconds for image of size ({},{},{})'.format(e - s, ima1.shape[0], ima1.shape[1], ima1.shape[2]))
flow_with_legend (of_brox)

Let's compute the DFD error:

In [ ]:
# https://github.com/opencv/opencv/blob/master/samples/python/opt_flow.py
def warp_flow(img, flow):
    h, w = img.shape[:2]
    flow = -flow
    flow[:,:,0] += np.arange(w)
    flow[:,:,1] += np.arange(h)[:,np.newaxis]

    compensated_ima = cv2.remap(img, flow, None, cv2.INTER_LINEAR)
    return compensated_ima

def DFD_flow(ima1, ima2, flow):
    '''
    Compute the Displaced Frame Difference between I2(t) and I1(t-1) as:
    I2(r,t) - I1(r-D(r), t-1)
    where D(r) is the optical flow
    '''
    ima1_comp = warp_flow(ima1, flow)
    #ima1_comp = warp_back(ima1, flow[:,:,0], flow[:,:,1])
    dfd   = ima2 - ima1_comp
    dfd_e = np.sqrt(np.sum(dfd*dfd)) / np.prod(ima1.shape[0:2])

    return dfd, dfd_e

In [ ]:
#DFD_br,   err_br   = DFD_flow(ima1g.astype(np.float32), ima2g.astype(np.float32), of_brox.astype(np.float32))
#print ('Brox 2004    DFD square error: {}'.format(err_br))
#norm_DFD_brox = (((DFD_br   - np.min(DFD_br))  / 510) * 255).astype(np.uint8)
#display_image (norm_DFD_brox, 'DFD Brox 2004', size=1.0)


of_null = np.zeros_like(of_raft)  # No optical flow 

FD, err_nc         = DFD_flow(ima1g.astype(np.float32), ima2g.astype(np.float32), of_null)
DFD_raft, err_raft = DFD_flow(ima1g.astype(np.float32), ima2g.astype(np.float32), of_raft)
DFD_hs,   err_hs   = DFD_flow(ima1g.astype(np.float32), ima2g.astype(np.float32), of_hs.astype(np.float32))
DFD_fb,   err_fb   = DFD_flow(ima1g.astype(np.float32), ima2g.astype(np.float32), of_fb)
DFD_br,   err_br   = DFD_flow(ima1g.astype(np.float32), ima2g.astype(np.float32), of_brox.astype(np.float32))

print ('------------  FD square error: {}'.format(err_nc))
print ('RAFT         DFD square error: {}'.format(err_raft))
print ('Horn-Schunck DFD square error: {}'.format(err_hs))
print ('Brox 2004    DFD square error: {}'.format(err_br))
print ('Farneback    DFD square error: {}'.format(err_fb))

norm_FD_null  = (((FD - np.min(FD))/ 510) * 255).astype(np.uint8)
norm_DFD_raft = (((DFD_raft - np.min(DFD_raft))/ 510) * 255).astype(np.uint8)
norm_DFD_hs   = (((DFD_hs   - np.min(DFD_hs))  / 510) * 255).astype(np.uint8)
norm_DFD_brox = (((DFD_br   - np.min(DFD_br))  / 510) * 255).astype(np.uint8)
norm_DFD_fb   = (((DFD_fb   - np.min(DFD_fb))  / 510) * 255).astype(np.uint8)

display_image (norm_FD_null,  'Frame Difference', size=1.0)
display_image (norm_DFD_raft, 'DFD RAFT', size=1.0)
display_image (norm_DFD_hs,   'DFD Horn-Schunck', size=1.0)
display_image (norm_DFD_brox, 'DFD Brox 2004', size=1.0)
display_image (norm_DFD_fb,   'DFD Farneback', size=1.0)


Let's simulate a camera movement two pixels to the right.
The motion vectors should be all $u = -2, v = 0$

In [ ]:
import cv2
import numpy as np

# Check
%cd /content
ima_ori = cv2.imread(image1_name)
dx = -2
dy = -0

h,w = ima_ori.shape[0:2]

# Create a pair of images simulating camera translational motion
ima1 = ima_ori[0:h-np.abs(dy), 0:w-np.abs(dx),:]
ima2 = ima_ori[np.abs(dy):,    np.abs(dx):   ,:]

# Add 1 pixel at the right
ima1 = cv2.copyMakeBorder(ima1,np.abs(dy),0,0,np.abs(dx),cv2.BORDER_REPLICATE)
ima2 = cv2.copyMakeBorder(ima2,np.abs(dy),0,0,np.abs(dx),cv2.BORDER_REPLICATE)


image1_name = '/content/cairo1_prev.png'
image2_name = '/content/cairo1_curr.png'
cv2.imwrite(image1_name, ima1)
cv2.imwrite(image2_name, ima2)


ima1g = cv2.cvtColor(ima1, cv2.COLOR_BGR2GRAY)
ima2g = cv2.cvtColor(ima2, cv2.COLOR_BGR2GRAY)

#ima1 = ima2.astype(np.float32)
#ima2 = ima2.astype(np.float32)

# Let's create the Ground Truth optical flow
gt_flow = np.stack([np.ones_like(ima1g)*dx, np.ones_like(ima2g)*dy], axis=2)

Now, compute optical flow with all the previous methods:

In [ ]:
# RAFT
%cd /content/RAFT
of_raft, tt, flow_up = inference(image1_name, image2_name, args) 

# Horn-Schunck
U, V = HornSchunck(ima1g, ima2g, alpha=1.0, Niter=100)
of_hs = np.stack([U,V], axis=2)

# Farneback
of_fb = cv2.calcOpticalFlowFarneback(ima1g,ima2g, None, 0.5, 3, 15, 3, 5, 1.2, 0) # Farneback

# Brox
%cd /content/pyflow
import pyflow
u, v, im2W = pyflow.coarse2fine_flow(
    ima1.astype(float)/255, ima2.astype(float)/255, alpha, ratio, minWidth, nOuterFPIterations, nInnerFPIterations,
    nSORIterations, colType)
of_brox = np.stack([u,v], axis=2)

Now, lets compute the average End Point Error metric:
$EPE = \|V_{gt} - V_{calc}\| = \sqrt{(\Delta x_{gt}-\Delta x_{calc})^2 + (\Delta y_{gt} - \Delta y_{calc})^2}$. This measures the 'correctness' of the motion vectors. Note that some methods (RAFT) are optimized for this criterion.


In [ ]:
import numpy as np
def EPE (gt_flow, hypo_flow):
  sqerr = (gt_flow-hypo_flow)**2
  return np.sum(np.sqrt(sqerr[0]+sqerr[1])) / gt_flow[0].size

In [ ]:
epe_raft = EPE(gt_flow, of_raft)

epe_hs   = EPE(gt_flow, of_hs)

epe_fb   = EPE(gt_flow, of_fb)

%cd /content/pyflow
epe_brox = EPE(gt_flow, of_brox)

print ('EPE RAFT:         {}'.format(epe_raft))
print ('EPE Horn-Schunck: {}'.format(epe_hs))
print ('EPE Farneback:    {}'.format(epe_fb))
print ('EPE Brox:         {}'.format(epe_brox))

Cumpute the DFD:

In [ ]:
DFD_raft, err_raft = DFD_flow(ima1.astype(np.float32), ima2.astype(np.float32), of_raft)
DFD_hs,   err_hs   = DFD_flow(ima1.astype(np.float32), ima2.astype(np.float32), of_hs.astype(np.float32))
DFD_fb,   err_fb   = DFD_flow(ima1.astype(np.float32), ima2.astype(np.float32), of_fb)
DFD_br,   err_br   = DFD_flow(ima1.astype(np.float32), ima2.astype(np.float32), of_brox.astype(np.float32))

print ('RAFT         DFD square error: {}'.format(err_raft))
print ('Horn-Schunck DFD square error: {}'.format(err_hs))
print ('Farneback    DFD square error: {}'.format(err_fb))
print ('Brox 2004    DFD square error: {}'.format(err_br))


In this case, where the motion is the same for all pixels and the displacement is small, the Farneback method provides a good compensated image.

## Comparison on SINTEL

[SINTEL](http://sintel.is.tue.mpg.de/) is a data set for the evaluation of optical flow derived from the open source 3D animated short film, Sintel. The key features are: very long sequences, large motions, specular reflections, motion blur, defocus blur and atmospheric effects.



In [ ]:
import numpy as np
import os
import sys

# Adapted from: https://stackoverflow.com/questions/28013200/reading-middlebury-flow-files-with-python-bytes-array-numpy
def read_flo(of_file_name):
    # WARNING: this will work on little-endian architectures (eg Intel x86) only!
    data2D = None
    ok     = False
    with open(of_file_name, 'rb') as f:
        magic = np.fromfile(f, np.float32, count=1)
        if 202021.25 != magic:
            print('Magic number incorrect. Invalid .flo file')
        else:
            w = np.fromfile(f, np.int32, count=1)[0]
            h = np.fromfile(f, np.int32, count=1)[0]
            print('Reading %d x %d flo file' % (w, h))
            data = np.fromfile(f, np.float32, count=2*w*h)
            # Reshape data into 3D array (columns, rows, bands)
            data2D = np.resize(data, (h, w, 2))
            ok = True
    return data2D, ok

In [ ]:
# RAFT
%cd /content/RAFT
of_raft, tt, flow_up = inference(image1_name, image2_name, args) 

# Horn-Schunck
U, V = HornSchunck(ima1g, ima2g, alpha=1.0, Niter=100)
of_hs = np.stack([U,V], axis=2)

# Farneback
of_fb = cv2.calcOpticalFlowFarneback(ima1g,ima2g, None, 0.5, 3, 15, 3, 5, 1.2, 0) # Farneback

# Brox
%cd /content/pyflow
import pyflow
u, v, im2W = pyflow.coarse2fine_flow(
    ima1.astype(float)/255, ima2.astype(float)/255, alpha, ratio, minWidth, nOuterFPIterations, nInnerFPIterations,
    nSORIterations, colType)
of_brox = np.stack([u,v], axis=2)

## Conclusions:

Depending of the type of motion and the type of metric, the methods behave differently. For instance, RAFT is optimized for EPE metric (the loss function is similar to EPE) and obtains better result for this metric. For simple motions, all methods work quite well. For complex motion (try computing optical flow between cd_000042.png and cd_000046.png), RAFT is slightly better but, for simpler motions, Farneback gives excellent results. Brox is also very good but takes a long time unless you use a GPU implementation (the one used here does not).

For image compensation, if your motion is not highly complex, classical methods are still an interesting choice.

For motion segmentation or action recognition, Brox and RAFT are good candidates as they capture better the transitions between the motion of different objects. 